## Dataset: Labeled Faces in the Wild ##
## Experiment: (experiment_1) Image based gender classification ##

In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import pandas as pd
import os
import numpy as np
# import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sattvik/envs/pytorch_env/lib/py

### Parameters ###

In [2]:
epochs = 40
batch_size = 64
learning_rate = 0.001
# momentum = 0.5
num_workers = 2

data_path = '../data/lfw/data/'

# default `log_dir` is "runs" - we'll be more specific here
!rm -rf ./runs/experiment_1
writer = SummaryWriter('runs/experiment_1')

### Dataset class used to instanciate the data loader ###

In [3]:
class LFWDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_path, attributes_df, transform=None):
        self.attributes_df = attributes_df
        self.data_path = data_path
        self.transform = transform

    def __len__(self):
        return len(self.attributes_df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_path, "lfw_home/lfw_funneled", self.attributes_df.iloc[idx]['person'].replace(' ', '_'),"{}_{:04d}.jpg".format(self.attributes_df.iloc[idx]['person'].replace(' ', '_'),self.attributes_df.iloc[idx]['imagenum']))
#         img = torch.from_numpy(cv2.imread(img_path))
        img = Image.open(img_path, mode='r')
        
        label = self.attributes_df.iloc[idx]['Male']>0
        
        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.float)

### Creating datasets ###

In [4]:
# read file
attributes_df = pd.read_csv(os.path.join(data_path,'lfw_attributes.txt'))

# split data into training val and test
all_names = attributes_df.person.unique()
tt_msk = np.random.rand(len(all_names)) < 0.8
temp_train_names = all_names[tt_msk]
tv_msk = np.random.rand(len(temp_train_names)) < 0.8
train_names = temp_train_names[tv_msk]
val_names = temp_train_names[~tv_msk]
test_names = all_names[~tt_msk]
del all_names, tt_msk, temp_train_names, tv_msk

# create train val and test dataframes
train_df = attributes_df.loc[attributes_df['person'].isin(train_names)]
val_df = attributes_df.loc[attributes_df['person'].isin(val_names)]
test_df = attributes_df.loc[attributes_df['person'].isin(test_names)]
del attributes_df, train_names, val_names, test_names

# create datasets
train_dataset = LFWDataset(data_path, train_df, transform=transforms.Compose([
#                                                     transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()
                                                    ]))
val_dataset = LFWDataset(data_path, val_df, transform=transforms.Compose([
#                                                     transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()]))
test_dataset = LFWDataset(data_path, test_df, transform=transforms.Compose([
#                                                     transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor()]))
del train_df, val_df, test_df

print(len(train_dataset), len(val_dataset), len(test_dataset))

8452 2017 2674


### Declaring dataloaders ###

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

print(len(train_dataloader), len(val_dataloader), len(test_dataloader))

133 32 42


### Sanity check ###

In [6]:
# for i , (img, label) in enumerate(train_dataloader):
#     for im, lab in zip(img, label):
#         plt.imshow(np.moveaxis(np.asarray(im), 0, -1))
#         plt.show()
#         print(lab)
#     if i>2:
#         break

### Define Model ###

In [7]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool1 = nn.MaxPool2d(4,4)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.pool2 = nn.MaxPool2d(4,4)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.pool3 = nn.MaxPool2d(2,2)
#         self.conv4 = nn.Conv2d(32, 32, 5)
        self.fc1 = nn.Linear(800, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 1)
        self.dropout_layer1 = nn.Dropout(p=0.6)
        self.dropout_layer2 = nn.Dropout(p=0.5)
#         self.dropout_layer3 = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
#         x = F.relu(self.conv4(x))
        x = x.view(x.shape[0],-1)
        x = self.dropout_layer1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout_layer2(x)
#         x = self.dropout_layer3(x)
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

In [8]:
model = ResNet().cuda()
# model = ResNet()
print(model)

ResNet(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=800, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (dropout_layer1): Dropout(p=0.6, inplace=False)
  (dropout_layer2): Dropout(p=0.5, inplace=False)
)


### All layers except the last are frozen ###

In [9]:
# print(model)

# def unfreeze_layer4():
#     for p in model.rnet.layer4.parameters():
#         p.requires_grad = True

# def unfreeze_layer3():
#     for p in model.rnet.layer3.parameters():
#         p.requires_grad = True

# def unfreeze_layer2():
#     for p in model.rnet.layer2.parameters():
#         p.requires_grad = True

# def unfreeze_layer1():
#     for p in model.rnet.layer1.parameters():
#         p.requires_grad = True

# for p in model.parameters():
#     p.requires_grad = False

# for p in model.rnet.fc.parameters():
#     p.requires_grad = True

# for p in model.parameters():
#     print(p.requires_grad)

# unfreeze_layer4()

### Define training and validation functions ###

In [10]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def perform_evaluation(val_model, dataloader):
    with torch.no_grad():
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # forward pass
            output = val_model(data)
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("testing loss: {} and testing accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

In [12]:
def perform_validation(val_model, dataloader):
    with torch.no_grad():
        epoch_loss = 0
        epoch_accuracy = 0
        for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
            # move data batch to GPU
            data = data.cuda()
            target = target.cuda()
            # forward pass
            output = val_model(data)
            loss = F.binary_cross_entropy(output, target.unsqueeze(1))
            # compute average loss an accuracy
            output = output.to('cpu')
            target = target.to('cpu')
            current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
            epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
            epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("val loss: {} and val accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

In [13]:
def perform_training(val_model, dataloader):
    epoch_loss = 0
    epoch_accuracy = 0
    for batch_idx, (data, target) in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
        # move data batch to GPU
        data = data.cuda()
        target = target.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        output = val_model(data)
#         print(output)
        loss = F.binary_cross_entropy(output, target.unsqueeze(1))
        # backward pass
        loss.backward()
        optimizer.step()
        # compute average loss an accuracy
        output = output.to('cpu')
        target = target.to('cpu')
        current_acc = torch.tensor(((output>0.5)== torch.tensor(target.unsqueeze(1), dtype=torch.bool)).sum(), dtype=torch.float)/torch.tensor(len(target), dtype=torch.float)
        epoch_loss = ((epoch_loss*batch_idx) + loss.item())/(batch_idx+1)
        epoch_accuracy = ((epoch_accuracy*batch_idx) + current_acc.item())/(batch_idx+1)
    print("train loss: {} and train accuracy: {}".format(epoch_loss, epoch_accuracy))
    return epoch_loss, epoch_accuracy

### Initiate training ###

In [14]:
train_losses=[]
train_accuracies=[]
val_losses=[]
val_accuracies=[]
max_val_acc = 0
for epoch in range(epochs):
    # run train and val epochs
    print("Epoch: {}".format(epoch))
    model.train()
    train_loss, train_acc = perform_training(model, train_dataloader)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    writer.add_scalar('training loss', train_loss, epoch)
    writer.add_scalar('training accuracy', train_acc, epoch)
    model.eval()
    val_loss, val_acc = perform_validation(model, val_dataloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    writer.add_scalar('validation loss', val_loss, epoch)
    writer.add_scalar('validation accuracy', val_acc, epoch)
    if val_acc > max_val_acc:
        print("saving model on epoch {}".format(epoch))
        torch.save(model.state_dict(), "models/best_model.pt")
writer.close()

Epoch: 0


/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



train loss: 0.5045932390187915 and train accuracy: 0.7785479323308271


/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app



val loss: 0.4686754746362567 and val accuracy: 0.7609493378549814
saving model on epoch 0
Epoch: 1


...


val loss: 0.2365908941719681 and val accuracy: 0.913603812456131
saving model on epoch 37
Epoch: 38



train loss: 0.18604470548549093 and train accuracy: 0.9325657894736842



val loss: 0.23793210415169597 and val accuracy: 0.916015625
saving model on epoch 38
Epoch: 39



train loss: 0.1895410345125019 and train accuracy: 0.9312734962406015



val loss: 0.22716396884061396 and val accuracy: 0.917998343706131
saving model on epoch 39


### Evaluate the model ###

In [15]:
# evaluate final model
model.eval()
test_loss, test_acc = perform_evaluation(model, test_dataloader)

# evaluate best model
eval_model = ResNet().cuda()
eval_model.load_state_dict(torch.load("models/best_model.pt"))
eval_model.eval()
best_test_loss, best_test_acc = perform_evaluation(eval_model, test_dataloader)

print("final model loss: {}, final model accuracy: {}, best model loss: {}, best model accuracy: {}".format(test_loss, test_acc, best_test_loss, best_test_acc))

/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app



testing loss: 0.24142404823076158 and testing accuracy: 0.906369047505515



testing loss: 0.24516284483529271 and testing accuracy: 0.9107738094670432
final model loss: 0.24142404823076158, final model accuracy: 0.906369047505515, best model loss: 0.24516284483529271, best model accuracy: 0.9107738094670432


In [16]:
torch.save(model.state_dict(), "final_model.pt")